In [0]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PostgreSQL to ADLS Gen2") \
    .config("spark.jars", "dbfs:/FileStore/tables/postgresql_42_7_4.jar") \
    .getOrCreate()

# PostgreSQL connection details
jdbc_url = "jdbc:postgresql://ep-icy-meadow-a8zfunjg.eastus2.azure.neon.tech:5432/firstDB"
db_table = "Customer_SCD2"
user = "firstDB_owner"
password = "qLmOcA3FMj8C"
driver = "org.postgresql.Driver"

# Load data from PostgreSQL into a DataFrame
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}


In [0]:

postgres_df = spark.read.format("jdbc").options(
    url=f"{jdbc_url}?user={user}&password={password}&sslmode=require",
    dbtable=db_table,
    driver=driver
).load()

# Display loaded DataFrame
postgres_df.show()



+-----------+--------+----------------+----------+----------+----------+
|Customer_id|    Name|         Address|Start_date|  End_date|Is_current|
+-----------+--------+----------------+----------+----------+----------+
|          1|  John D|           Paris|2024-12-05|      NULL|      true|
|          4|   Jerry|        New york|2024-12-05|      NULL|      true|
|          1|  John D|           Paris|2024-12-05|2024-12-05|     false|
|          3|     Joe|  36 D, New york|2024-12-05|      NULL|      true|
|          1|Johny D1|xyz street Paris|2024-12-05|      NULL|      true|
+-----------+--------+----------------+----------+----------+----------+



In [0]:
def mount_adls(storage_name, container_name):
    # get secrets from key vault
    clinet_id = dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-client-id")
    tenant_id = dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-tenant-id")
    secreat_id =  dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-secret-key")

    # set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": clinet_id,
          "fs.azure.account.oauth2.client.secret": secreat_id,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # unmount the mount points if is there any exists
    if any(mount.mountPoint == f"/mnt/{storage_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_name}/{container_name}")
    
    # Manage storage account container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_name}/{container_name}",
        extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls('zecdatastorage', 'presentation')

/mnt/zecdatastorage/presentation has been unmounted.


mountPoint,source,encryptionType
/mnt/zecdatastorage/demo,abfss://demo@zecdatastorage.dfs.core.windows.net/,
/databricks-datasets,databricks-datasets,
/mnt/zecdatastorage/processed,abfss://processed@zecdatastorage.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/zecdatastorage/presentation,abfss://presentation@zecdatastorage.dfs.core.windows.net/,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/zecdatastorage/raw,abfss://raw@zecdatastorage.dfs.core.windows.net/,
/Volume,DbfsReserved,


In [0]:
adls_path = '/mnt/zecdatastorage/presentation'
postgres_df.write.mode("overwrite").format("parquet").save(adls_path)

print(f"Data written successfully to {adls_path}")



Data written successfully to /mnt/zecdatastorage/presentation


In [0]:
# Stop SparkSession
spark.stop()



com.databricks.backend.common.rpc.SparkStoppedException: Spark down: 
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommandAndGetError(DriverWrapper.scala:693)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:730)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$runInnerLoop$1(DriverWrapper.scala:560)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.backend.daemon.driver.DriverWrapper.withAttributionContext(DriverWrapper.scala:72)
	at com.databricks.ba